In [3]:
#!/usr/bin/env python3
"""
rename_notebooks.py

Cari file notebook yang namanya dimulai dengan 'Untitled' (mis. Untitled.ipynb, Untitled1.ipynb, Untitled (1).ipynb)
dan ubah namanya menjadi <nama-folder>.ipynb. Jika nama target sudah ada, tambahkan suffix _1, _2, dst.
Default: dry-run (hanya menampilkan rencana). Gunakan --apply untuk benar-benar mengganti nama.

Abaikan folder: .git, __pycache__, .ipynb_checkpoints
Aman dijalankan di Jupyter karena menggunakan parse_known_args().
"""
import os
import argparse
import re
import sys
from pathlib import Path

IGNORED_DIRS = {'.git', '__pycache__', '.ipynb_checkpoints'}

def find_notebooks(root):
    matches = []
    pattern = re.compile(r'^Untitled.*\.ipynb$', re.IGNORECASE)
    for dirpath, dirnames, filenames in os.walk(root):
        # skip ignored dirs
        dirnames[:] = [d for d in dirnames if d not in IGNORED_DIRS]
        for f in filenames:
            if pattern.match(f):
                matches.append(Path(dirpath) / f)
    return matches

def unique_target_for_planning(src_path: Path, base_name: str, used_targets: set):
    """
    Pilih nama target yang unik, mempertimbangkan file yang sudah ada di filesystem
    dan target yang sudah direncanakan (used_targets).
    """
    parent = src_path.parent
    stem = Path(base_name).stem
    suffix = Path(base_name).suffix or ".ipynb"
    candidate = base_name
    i = 0
    # keep incrementing until candidate is not present and not already planned
    while (parent / candidate).exists() or str(parent / candidate) in used_targets:
        i += 1
        candidate = f"{stem}_{i}{suffix}"
    return parent / candidate

def plan_renames(root):
    notebooks = find_notebooks(root)
    planned = []
    used_targets = set()
    for nb in notebooks:
        folder_name = nb.parent.name or 'root'
        target_name = f"{folder_name}.ipynb"
        target = unique_target_for_planning(nb, target_name, used_targets)
        # if source already equals target (already correctly named), skip
        try:
            if nb.resolve() == target.resolve():
                continue
        except FileNotFoundError:
            # in case resolution fails, ignore and proceed
            pass
        planned.append((nb, target))
        used_targets.add(str(target))
    return planned

def do_rename(planned):
    for src, dst in planned:
        dst.parent.mkdir(parents=True, exist_ok=True)
        src.rename(dst)
        print(f"Renamed: {src} -> {dst}")

def main(argv=None):
    parser = argparse.ArgumentParser(description="Rename Untitled*.ipynb to <folder>.ipynb (default: dry-run).")
    parser.add_argument('--root', default='.', help='Root folder to search (default: current dir)')
    parser.add_argument('--apply', action='store_true', help='Actually rename the files. If not set, script runs in dry-run mode.')

    # ignore extra args added by ipykernel/jupyter (mis. -f <path>)
    if argv is None:
        args, _unknown = parser.parse_known_args()
    else:
        args = parser.parse_args(argv)

    planned = plan_renames(args.root)
    if not planned:
        print("Tidak ditemukan file Untitled*.ipynb atau tidak ada perubahan yang diperlukan.")
        return 0

    print("Rencana penggantian nama:")
    for src, dst in planned:
        print(f"  {src} -> {dst}")

    if not args.apply:
        print("\nDry-run selesai. Untuk menerapkan perubahan, jalankan skrip dengan opsi --apply")
        return 0

    do_rename(planned)
    print(f"\nSelesai. {len(planned)} file diubah.")
    return 0

if __name__ == '__main__':
    sys.exit(main())

Rencana penggantian nama:
  Untitled.ipynb -> root.ipynb
  Untitled1.ipynb -> root_1.ipynb

Dry-run selesai. Untuk menerapkan perubahan, jalankan skrip dengan opsi --apply


SystemExit: 0